In [39]:
from pytube import YouTube
import torch
from utils.general import (check_file, check_img_size, cv2, non_max_suppression)

In [90]:
yt = YouTube('https://www.youtube.com/watch?v=WfuE61ZkuRo')
video = yt.streams.filter(file_extension='mp4').first()
video.download('video/')

'/home/theo/Desktop/flasktuto/AppBrandSeeker/video/Faire GLISSER un BUS  ÇA FAIT QUOI .mp4'

In [41]:
from pytube import YouTube
def pytube_dl(url):
    yt = YouTube(str(url))
    video = yt.streams.filter(file_extension='mp4').first()
    video.download('video/')


In [91]:
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages

In [92]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='weights/best.pt')

stride, names, pt = model.stride, model.names, model.pt
imgsz = check_img_size((640, 640), s=stride)  

Using cache found in /home/theo/.var/app/com.visualstudio.code/cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-7-4 Python-3.10.4 torch-1.12.0+cu102 CPU

Fusing layers... 
Model summary: 213 layers, 7112611 parameters, 0 gradients
Adding AutoShape... 


In [121]:
dataset = LoadImages("video/", img_size=imgsz, stride=stride, auto=pt, only_vids=True)

In [122]:
bs = 1

In [123]:
brand_count = {}

In [124]:
path, im, im0s, vid_cap, s, frame = dataset.__iter__().__next__()


In [125]:
initial_framerate = vid_cap.get(cv2.CAP_PROP_FPS)
real_framerate = initial_framerate / round(initial_framerate / 15)
total_frames = dataset.frames
dataset.frame = 0

In [126]:
total_frames = dataset.frames

In [127]:
from tqdm.autonotebook import tqdm

In [ ]:
i=0
for path, im, im0s, vid_cap, s, frame in tqdm(dataset, total=total_frames):
    i+=1
    print(i)


In [128]:
from utils.torch_utils import select_device, time_sync

In [129]:
device = select_device('')

YOLOv5 🚀 2022-7-4 Python-3.10.4 torch-1.12.0+cu102 CPU



In [130]:
dt, seen = [0.0, 0.0, 0.0], 0

In [144]:
framerate = 2

In [145]:
from utils.filtering import filter_output
from utils.pdf_generator import pdf_generator, normalize

In [146]:
save_dir = "saving_predict"

In [147]:
save_unprocessed_output = False

In [148]:
pred_timing_start = time_sync()

In [149]:
for path, im, im0s, vid_cap, s, frame in tqdm(dataset, total=total_frames):

        # skip the frame if it isn't in the specified framerate
        if frame % round(initial_framerate / framerate) != 0:
            continue

        t1 = time_sync()
        im = torch.from_numpy(im).to(device)
        im = im.float()
        im /= 255  # 0 - 255 to 0.0 - 1.0

        if len(im.shape) == 3:
            im = im[None]  # expand for batch dim
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        pred = model(im)
        t3 = time_sync()
        dt[1] += t3 - t2

        # NMS
        pred = non_max_suppression(pred, conf_thres=0.35, max_det=5)
        pred = pred[0].tolist()
        dt[2] += time_sync() - t3

        has_prediction = len(pred)
        if has_prediction:
            for brand in pred:
                label = names[int(brand[5])]

                # Retrieve or create a dictionnary key for the label and add the bbox, confidence and frame of the prediction
                brand_count[label] = brand_count.get(label, {"bbox": [], "confidence": [], "frame": []})
                brand_count[label]["bbox"].append(brand[0:4])
                brand_count[label]["confidence"].append(brand[4])
                brand_count[label]["frame"].append(frame)
    
    # Generate an output if a prediction has been made
if brand_count:
    filtered_output = filter_output(brand_count, framerate)
    pdf_generator(path, filtered_output, save_dir)
    # cv2.imshow(im, framerate)

    if save_unprocessed_output:
        with open(f"{save_dir}/{normalize(path)}.txt", "w") as f:
            f.write(str(brand_count))
else:
    print("No prediction has been made")


pred_timing_stop = time_sync()
pred_timing = pred_timing_stop - pred_timing_start
print("Pred took %.2fs (%.2ffps)" % (pred_timing, ((total_frames / initial_framerate) * real_framerate) / pred_timing))


  0%|          | 0/35128 [00:00<?, ?it/s]


ZeroDivisionError: float division by zero